# Various tools to edit or add to a static master JSON.

### Add iscrowd to static json

In [162]:
import json
import pandas as pd


In [166]:
json_inpath = 'master.json'
json_outpath = 'master_adjusted.json'

# Load JSON object into a variable
# Add 'iscrowd: 0' into all annotations
# Save JSON object into file

In [167]:
# Open JSON
f = open(json_inpath)

# Load JSON
master_json = json.load(f)
annot = master_json['annotations']

# Add 'iscrowd' into all annotations
for i in annot:
    
    i['iscrowd'] = 0


In [168]:
# Save updated JSON object to new file
with open(json_outpath, "w") as outfile:
    json.dump(master_json, outfile)

## Add Area

In [80]:
import json
from shapely.geometry import Polygon

In [81]:
# Open JSON

json_inpath = 'master.json'
json_outpath = 'master.json'

f = open(json_inpath)

# Load JSON
master_json = json.load(f)

In [82]:
# iterate each annotation in json
for annotation in master_json['annotations']:
    polygon = []

    #iterate indiviual coordinates
    for index, coordinate in enumerate(annotation['segmentation'][0]):
        #if x or y point
        if index % 2 == 0:
            x = coordinate
        else:
            y = coordinate
            polygon.append((x, y))

    polygon = Polygon(polygon)
    annotation['area'] = int(polygon.area)


In [83]:
# Save updated JSON object to new file
with open(json_outpath, "w") as outfile:
    json.dump(master_json, outfile)

# Add Height and Width

In [155]:
import json
from PIL import Image

In [156]:
json_inpath = 'master.json'
json_outpath = 'master_adjusted.json'
images_root = "../../LEGACY_test/data/"

f = open(json_inpath)

# Load JSON
master_json = json.load(f)

In [157]:
for image in master_json['images']:
    img = Image.open(f"/home/beanbagthomas/code/hrf/datasets/LEGACY_test/data/{image['file_name']}")
    if image['height'] != img.height or image['width'] != img.width:
        print("NOT SAME")
    image['height'] = img.height
    image['width'] = img.width


In [31]:
# Save updated JSON object to new file
with open(json_outpath, "w") as outfile:
    json.dump(master_json, outfile)

# Pad edges of mask and bbox so they arent pixel edges

In [71]:
import json

In [72]:
json_inpath = 'master.json'
json_outpath = 'master_adjusted.json'

f = open(json_inpath)

# Load JSON
master_json = json.load(f)

In [73]:

#go through all annotations
for annot in master_json['annotations']:

    #find matching image and extract height and width
    image_found = False
    for image in master_json['images']:
        if annot['image_id'] == image['id']:
            width_x = image['width']
            height_y = image['height']
            image_found = True
            break

    if image_found == False:
        print("ERROR IMAGE NOT FOUND: ", annot['image_id'])
        break

    #go through each coordinate and adjust
    for index, coor in enumerate(annot['segmentation'][0]):
        # TURNS SEGM POINTS TO INTS
        annot['segmentation'][0][index] = int(coor)

        #raise zeros to 2
        if coor >= -1 and coor <= 1:
            annot['segmentation'][0][index] = 2
        
        #if coordinate is x point
        if index % 2 == 0:
            #if x point is at farthest edge adjust +2
            if coor > width_x - 2:
                annot['segmentation'][0][index] = width_x - 2

        #if coordinate is y point
        else:
            #if y point is at farthest edge adjust +2
            if coor > height_y - 2:
                annot['segmentation'][0][index] = height_y - 2

    #recalculate bbox
    lowest_x = min(point for index, point in enumerate(annot['segmentation'][0]) if index % 2 == 0)
    highest_x = max(point for index, point in enumerate(annot['segmentation'][0]) if index % 2 == 0)
    lowest_y = min(point for index, point in enumerate(annot['segmentation'][0]) if index % 2 != 0)
    highest_y = max(point for index, point in enumerate(annot['segmentation'][0]) if index % 2 != 0)
    
    annot['bbox'] = [lowest_x, lowest_y, highest_x-lowest_x, highest_y-lowest_y]


        




In [74]:
# Save updated JSON object to new file
with open(json_outpath, "w") as outfile:
    json.dump(master_json, outfile)

# Change Categories (used for flipping indiviual COCO JSON's where cat id's are mismatched)


In [141]:
import json

In [145]:
json_inpath = '/home/beanbagthomas/code/hrf/datasets/LEGACY_test/jsons/labels_raw.json'
json_outpath = 'labels.json'


f = open(json_inpath)

# Load JSON
master_json = json.load(f)

correct = {1:"Officer", 2:"Civilian", 3:"Riot Shield", 4:"Gun", 5:"Pepper Spray", 6:"Baton", 7:"Chemical Smoke", 8:"Blade"}

In [146]:
# get incorrect category

for cat in master_json['categories']:

    #if incorrect category
    if correct[cat['id']] != cat['name']: 
        wrong_id = cat['id']
        right_id = [key for key,val in correct.items() if val == cat['name']][0]

        print("wrong category: ", cat)
        print("right id: ", right_id)

        #go through all annotations
        for annot in master_json['annotations']:
            
            #if incorrect annotation
            if annot['category_id'] == wrong_id:
                # temporarily store cat_id in higher number to avoid cross contamination
                annot['category_id'] = right_id * 10000

        #change incorrect category
        cat['id'] = right_id

# change temp cat_ids to real
for annot in master_json['annotations']:
    if annot['category_id'] >= 10000:
        annot['category_id'] = int(annot['category_id'] / 10000)


In [147]:
# Save updated JSON object to new file
with open(json_outpath, "w") as outfile:
    json.dump(master_json, outfile)

# Add info & licenses